# Глава 2. Распознавание и обработка временных рядов

In [1]:
import numpy as np
import pandas as pd

## Учебная дазача: Получение временных рядов

### Загрузим данные

In [2]:
year_joined = pd.read_csv("data_nilsen/year_joined.csv")
emails = pd.read_csv("data_nilsen/emails.csv")
donations = pd.read_csv("data_nilsen/donations.csv")

Сначала в имеюшихся данных надо определить временные оси.

### DataFrame year_joined

In [3]:
year_joined.head()

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018


Надо увостовериться, что текущий статус пользователя определяется его годовым, а не другим, более краткосрочным статусом.

Для этого смотрим, сколько записей имеет каждый пользователь:

In [4]:
year_joined.groupby('user').count()

,userStats,yearJoined
user,,
0,1,1
1,1,1
2,1,1
3,1,1
4,1,1
...,...,...
995,1,1
996,1,1
997,1,1


In [5]:
year_joined.groupby('user').count().groupby('userStats').count()

,yearJoined
userStats,
1,1000


Результат:
- Всего имеет 1000 пользователей.
- У каждого из них по одной записи.

### DataFrame emails

In [6]:
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29 00:00:00
1,2.0,1.0,2015-07-13 00:00:00
2,2.0,1.0,2015-07-20 00:00:00
3,3.0,1.0,2015-07-27 00:00:00
4,1.0,1.0,2015-08-03 00:00:00


Преобразуем строки в колонке week в формат datetime

In [22]:
emails.week = pd.to_datetime(emails.week)
emails.head()

,emailsOpened,user,week
0,3.0,1.0,2015-06-29
1,2.0,1.0,2015-07-13
2,2.0,1.0,2015-07-20
3,3.0,1.0,2015-07-27
4,1.0,1.0,2015-08-03


Проверим, существуют ли пустые недели, когда пользователь не открыл ни одного электронного письма:

In [23]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


Ага. Пусто. Здесь у нас два варианта: либо пустые недели не регистрируются, либо пользователи открывают каждую неделю хотя бы одно электронное письмо.

Посмотрим историю работы с электронной почтой одного пользователя:

In [24]:
emails[emails.user == 998].head(15)

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


Результат: отдельные недели не указаны в списке.

Посчитаем количество еженедельных наблюдений между первым и последним событиями:

In [25]:
max_week = max(emails[emails.user == 998].week)
max_week

Timestamp('2018-05-28 00:00:00')

In [26]:
min_week = min(emails[emails.user == 998].week)
min_week

Timestamp('2017-12-04 00:00:00')

In [28]:
(max_week - min_week).days / 7

25.0

Между начальной и конечной учетной записью должно пройти 26 недель (25 + 1)

А сколько недель заданы для пользователя 998?

In [29]:
emails[emails.user == 998].shape

(24, 3)

Ага! 24, а должно быть 26...
Нам удалось выяснить, что в наборе есть недоствющие данные, для некоторых недель они не указаны.

Заполним недостающие недели:
- для этого сгенерируем объект Multiindex из нашего датафрейма, который представляет все комбинации недель и пользователей организации (декартово произведение);
- используем полученный индекс для переиндексации исходной таблицы и заполнения пропущенных значений нулем;

In [33]:
complete_idx = pd.MultiIndex.from_product((set(emails.week), set(emails.user)))
complete_idx.shape

(93247,)

In [34]:
complete_idx[:10]

MultiIndex([('2016-12-19',  1.0),
            ('2016-12-19',  3.0),
            ('2016-12-19',  5.0),
            ('2016-12-19',  6.0),
            ('2016-12-19',  9.0),
            ('2016-12-19', 10.0),
            ('2016-12-19', 14.0),
            ('2016-12-19', 16.0),
            ('2016-12-19', 20.0),
            ('2016-12-19', 21.0)],
           )

In [39]:
all_email = emails.set_index(['week', 'user']).reindex(complete_idx, fill_value = 0).reset_index()

In [40]:
all_email.head()

,level_0,level_1,emailsOpened
0,2016-12-19,1.0,3.0
1,2016-12-19,3.0,0.0
2,2016-12-19,5.0,0.0
3,2016-12-19,6.0,3.0
4,2016-12-19,9.0,3.0


Переименуем столбцы:

In [43]:
all_email.columns = ['week', 'user', 'emailsOpened']
all_email.head()

,week,user,emailsOpened
0,2016-12-19,1.0,3.0
1,2016-12-19,3.0,0.0
2,2016-12-19,5.0,0.0
3,2016-12-19,6.0,3.0
4,2016-12-19,9.0,3.0
